In [1]:
import requests
import csv
import time
import re
from typing import List, Dict
import json

from scenarios_model import scenarios_json, json2prompt 

In [2]:
# Authentic Scenario Prompts for Direction Discovery

starting_prompt = """
Generate prompts that establish genuine, unguarded conversations where people speak honestly without social filtering. Each scenario should feel natural and encourage authentic responses rather than polite assistant-like behavior.

**Format for each scenario:**
- Brief context setup
- Character descriptions that justify honest speech
- 20 conversation starters that would naturally elicit genuine opinions/reactions
"""

scenarios = [json2prompt(scenario) for scenario in scenarios_json]

output_format = """
## Output Format

For the scenario, provide exactly 20 prompts in a JSON format:

```
{
    "scenario": "{scenario}",
    "prompts": [
        "prompt1",
        "prompt2",
        ...
        "prompt20"
    ]
}
```

Ensure variety within each category and avoid repetitive phrasing.
"""

scenario_names = re.findall(r'## Scenario: (.*)', '\n'.join(scenarios))


In [3]:
import random
def call_gemini_api(prompt: str, api_key: str) -> str:
    """
    Call Gemini 2.0 Flash API via Google AI Studio with the given prompt
    """
    temp = max(0.3, min(1.0, random.gauss(0.7, 0.15)))
    
    # top_p: mean 0.9, variance ±0.05, clamp 0.8–0.95
    top_p = max(0.8, min(0.95, random.gauss(0.9, 0.03)))
    url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"
    
    headers = {
        'Content-Type': 'application/json',
        'x-goog-api-key': api_key
    }
    
    data = {
        "contents": [
            {
                "parts": [
                    {
                        "text": prompt
                    }
                ]
            }
        ],
        "generationConfig": {
            "temperature": temp,
            "topP": top_p,
            "responseMimeType": "application/json",
            "responseSchema": {
            "type": "OBJECT",
            "properties": {
                "scenario": { "type": "STRING" },
                "prompts": {
                "type": "ARRAY",
                "items": { "type": "STRING" }
                }
            },
            "required": ["scenario", "prompts"],
            "propertyOrdering": ["scenario", "prompts"]
            }
        }
    }
    
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        
        result = response.json()
        if 'candidates' in result and len(result['candidates']) > 0:
            return result['candidates'][0]['content']['parts'][0]['text']
        else:
            print(f"No candidates in response: {result}")
            return ""
            
    except requests.exceptions.RequestException as e:
        print(f"API request failed: {e}")
        return ""
    except (KeyError, IndexError) as e:
        print(f"Error parsing response: {e}")
        return ""

In [4]:
def parse_prompts_from_response(response_text: str, category_name: str) -> List[Dict[str, str]]:
    """
    Parse the JSON from Gemini's response and return as list of dicts
    """
    prompts = []
    response_json = json.loads(response_text)
    category_name = response_json["scenario"]
    for prompt in response_json["prompts"]:
        prompts.append({
            "category": category_name,
            "prompt": prompt
        })
    return prompts

def generate_category_prompts(scenario_prompt: str, scenario_name: str, api_key: str, target_count: int = 100) -> List[Dict[str, str]]:
    """
    Generate prompts for a specific category using batched API calls
    """
    all_prompts = []
    batch_size = 20  # Generate 20 prompts per API call
    
    print(f"Generating prompts for category: {scenario_name}")
    
    while len(all_prompts) < target_count:
        remaining = target_count - len(all_prompts)
        current_batch_size = min(batch_size, remaining)
        
        # Create the full prompt
        full_prompt = starting_prompt + scenario_prompt.replace("20", str(current_batch_size)) + output_format
        
        print(f"  Requesting {current_batch_size} prompts (total so far: {len(all_prompts)})")
        
        # Call the API
        response = call_gemini_api(full_prompt, api_key)
        
        if response:
            # Parse the response
            new_prompts = parse_prompts_from_response(response, scenario_name)
            all_prompts.extend(new_prompts)
            print(f"  Got {len(new_prompts)} prompts")
        else:
            print(f"  Failed to get response for batch")
            break
        
        # Add a small delay to be respectful to the API
        time.sleep(1)

    # Trim to exact target count if we got more
    return all_prompts[:target_count]

In [ ]:
def save_prompts_to_csv(all_prompts: List[Dict[str, str]], filename: str, append: bool = True):
    """
    Save all prompts to a CSV file (append mode by default)
    """
    import os
    
    file_exists = os.path.exists(filename)
    mode = 'a' if append and file_exists else 'w'
    
    print(f"File exists: {file_exists}, Mode: {mode}")
    print(f"{'Appending' if mode == 'a' else 'Writing'} {len(all_prompts)} prompts to {filename}")
    
    with open(filename, mode, newline='', encoding='utf-8') as csvfile:
        fieldnames = ['category', 'prompt']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Only write header if we're in write mode (new file or overwrite)
        if mode == 'w':
            writer.writeheader()
            
        for prompt_data in all_prompts:
            writer.writerow(prompt_data)
    
    print(f"Successfully {'appended to' if mode == 'a' else 'saved to'} {filename}")

In [ ]:
def generate_evaluation_dataset(api_key: str, prompts_per_category: int = 20):
    """
    Generate the complete evaluation dataset
    """
    all_prompts = []
    
    print(f"Starting generation of evaluation dataset with {prompts_per_category} prompts per category")
    print("=" * 60)
    
    for i, (scenario_prompt, scenario_name) in enumerate(zip(scenarios, scenario_names)):
        print(f"\nProcessing category {i+1}: {scenario_name}")
        
        try:
            category_prompts = generate_category_prompts(
                scenario_prompt, 
                scenario_name, 
                api_key, 
                target_count=prompts_per_category
            )
            
            all_prompts.extend(category_prompts)
            print(f"✓ Successfully generated {len(category_prompts)} prompts for {scenario_name}")
            
        except Exception as e:
            print(f"✗ Error generating prompts for {scenario_name}: {e}")
            continue
    
    print("\n" + "=" * 60)
    print(f"Generation complete! Total prompts: {len(all_prompts)}")
    
    # Save to CSV (append mode)
    save_prompts_to_csv(all_prompts, 'scenarios_uncleaned.csv', append=True)
    
    return all_prompts


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv("../.env")

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if __name__ == "__main__":
    api_key = GOOGLE_API_KEY
    
    prompts = generate_evaluation_dataset(api_key, prompts_per_category=20)




Starting generation of evaluation dataset with 20 prompts per category

Processing category 1/5: Stoned Roommates Chilling
Generating prompts for category: Stoned Roommates Chilling
  Requesting 20 prompts (total so far: 0)
  Got 20 prompts
✓ Successfully generated 20 prompts for Stoned Roommates Chilling

Processing category 2/5: Drunk Friends at 3AM
Generating prompts for category: Drunk Friends at 3AM
  Requesting 20 prompts (total so far: 0)
  Got 20 prompts
✓ Successfully generated 20 prompts for Drunk Friends at 3AM

Processing category 3/5: Anonymous Online Forum
Generating prompts for category: Anonymous Online Forum
  Requesting 20 prompts (total so far: 0)
  Got 20 prompts
✓ Successfully generated 20 prompts for Anonymous Online Forum

Processing category 4/5: Therapy Session (Internal Monologue)
Generating prompts for category: Therapy Session (Internal Monologue)
  Requesting 20 prompts (total so far: 0)
  Got 20 prompts
✓ Successfully generated 20 prompts for Therapy Sessi